## Decision Tree - Classification

In [12]:
import sys, types, os;has_mfs = sys.version_info > (3, 5);p ...<rest of file>

SyntaxError: invalid syntax (Temp/ipykernel_4156/1095747927.py, line 1)

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#plt.rcParams["figure.figsize"] = (10,6)
import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

ImportError: cannot import name 'style' from 'matplotlib' (C:\Users\yavuz\anaconda3\lib\site-packages\matplotlib\__init__.py)

In [ ]:
#pip install xgboost

In [ ]:
df = pd.read_csv("penguins_size.csv")
df.head()

## Exploratory Data Analysis and Visualization

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
10/344

In [ ]:
df = df.dropna()

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df["sex"].unique()

In [ ]:
df["island"].unique()

In [ ]:
df["species"].unique()

In [ ]:
df[df["sex"]== "."]

In [ ]:
df[df.species =="Gentoo"].groupby("sex").describe().T

In [ ]:
df.loc[336, "sex"] = "MALE"

In [ ]:
df.loc[336, "sex"]

## Visualization

In [ ]:
df["species"].value_counts()

In [ ]:
sns.countplot(x="species", data = df);

In [ ]:
sns.catplot(x='species', data=df, kind='count',col='sex')

In [ ]:
plt.figure(figsize=(12,6))
sns.pairplot(df,hue='species',palette='Dark2')

In [ ]:
df.species.unique()

In [ ]:
##pip install plotly

In [ ]:
import plotly.express as px

In [ ]:
fig = px.scatter_3d(df, 
                    x='culmen_length_mm',
                    y='flipper_length_mm',
                    z='culmen_depth_mm',
                    color='species')
fig.show();

In [ ]:
df.select_dtypes("number")  #numerik olan verileri seçiyoruz

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(df.select_dtypes("number").corr(),annot=True, cmap='viridis')
plt.title("Correlation Matrix")

plt.show()
#kanat uzunluğuyla ağarlık arasında corr olduğunu seçebiliyoruz

## Feature Engineering

In [ ]:
df.drop("species", axis =1)  #türleri düşürüp get dummyies 

In [ ]:
pd.get_dummies(df.drop("species", axis =1), drop_first =True)  #düşürüp bu şekilde sonuç aldık

## Train | Test Split

In [ ]:
X = pd.get_dummies(df.drop('species',axis=1),drop_first=True)
y = df['species']                                          #model verilen ağırlandırdığı bir şeye bakmaz scale etmeye gerek yok

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# Modelling - Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
DT_model = DecisionTreeClassifier(random_state=101)

In [ ]:
DT_model.fit(X_train,y_train)

In [ ]:
y_pred = DT_model.predict(X_test)

## Model Performance on Classification Tasks

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report,plot_confusion_matrix

In [ ]:
def eval_metric(model, X_train, y_train, X_test, y_test):
    y_train_pred = model.predict(X_train)
    y_pred = model.predict(X_test)
    
    print("Test_Set")
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print()
    print("Train_Set")
    print(confusion_matrix(y_train, y_train_pred))
    print(classification_report(y_train, y_train_pred))

In [ ]:
plot_confusion_matrix(DT_model,X_test,y_test)

In [ ]:
from yellowbrick.classifier import ClassPredictionError

visualizer = ClassPredictionError(DT_model)

# Fit the training data to the visualizer
visualizer.fit(X_train, y_train)

# Evaluate the model on the test data
visualizer.score(X_test, y_test)

# Draw visualization
visualizer.poof();  #yapılan hataların hangi sınıf üzerinde yapıldığını söylüyor

In [ ]:
eval_metric(DT_model, X_train, y_train, X_test, y_test) #tres ve boostingte sıkıntı olmaz

## Cross Validate

In [ ]:
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.metrics import make_scorer

In [ ]:
model = DecisionTreeClassifier(max_depth=None, random_state=101)
#tüm scorelar birbirine yakın olduğu için micro takıldık 

scores = cross_validate(model, X_train, y_train, scoring = ["accuracy", "precision_micro", "recall_micro", "f1_micro"], cv = 10)
df_scores = pd.DataFrame(scores, index = range(1, 11))
df_scores

In [ ]:
df_scores.mean()[2:] #bütün scoreler dengeliyse micro dengesizlik varda macro sayısı 

## Feature Importances

In [ ]:
DT_model.feature_importances_    #
#Accuracy scorelara bakılır:
#balance micro
#inbalance macro
#inbalance ama (sayısı fazla olan istersek) weighted  #targıtları doğru tahmin etmek için önemini söylüyor

In [ ]:
df_f_i = pd.DataFrame(index=X.columns, data = DT_model.feature_importances_, 
                      columns = ["Feature Importance"]).sort_values("Feature Importance")
df_f_i  #data frame dönüştürüyoruz feature importance

In [ ]:
sns.barplot(x = df_f_i.index, y = 'Feature Importance', data = df_f_i)
plt.xticks(rotation = 90)
plt.tight_layout()

## Drop most important feature

In [ ]:
X2 = X.drop(columns = ["flipper_length_mm"]) 
# The feature that weighs too much on the estimate can sometimes cause overfitting. For this reason, the most important feature can be dropped and the scores can be checked again

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y, test_size=0.3, random_state=101)

In [ ]:
DT2_model = DecisionTreeClassifier(random_state=101)

In [ ]:
DT2_model.fit(X_train2, y_train2) 

In [ ]:
eval_metric(DT2_model, X_train2, y_train2, X_test2, y_test2)

In [ ]:
model = DecisionTreeClassifier(random_state=101)

scores = cross_validate(model, X_train2, y_train2, scoring = ["accuracy", "precision_micro", "recall_micro", "f1_micro"], cv = 10)
df_scores = pd.DataFrame(scores, index = range(1, 11))
df_scores

In [ ]:
df_scores.mean()[2:]

In [ ]:
from sklearn.metrics import make_scorer

In [ ]:
scoring = {'precision-Adelie': make_scorer(precision_score,  average=None, labels=["Adelie"]),
           'recall-Adelie': make_scorer(recall_score, average=None, labels =["Adelie"]),
           'f1-Adelie': make_scorer(f1_score, average=None, labels = ["Adelie"]),
          
          'precision-Chinstrap': make_scorer(precision_score,  average=None, labels=["Chinstrap"]),
          'recall-Chinstrap': make_scorer(recall_score, average=None, labels=["Chinstrap"]),
          'f1-Chinstrap': make_scorer(f1_score, average=None, labels=["Chinstrap"]),
          
          
          'precision-Gentoo': make_scorer(precision_score,  average=None, labels=["Gentoo"]),
          'recall-Gentoo': make_scorer(recall_score, average=None, labels = ["Gentoo"]),
          'f1-Gentoo': make_scorer(f1_score, average=None, labels = ["Gentoo"]),
          
          }

In [ ]:
model = DecisionTreeClassifier(max_depth=None, random_state=101)

scores = cross_validate(model, X_train2, y_train2, scoring = scoring, cv = 10)
df_scores = pd.DataFrame(scores, index = range(1, 11))
df_scores.mean()[2:]

## Visualize the Tree

In [ ]:
from sklearn.tree import plot_tree

In [ ]:
plt.figure(figsize=(12,8))
plot_tree(DT_model);

In [ ]:
plt.figure(figsize=(12,8),dpi=100)
plot_tree(DT_model, filled=True, feature_names=X.columns, class_names=df.species.unique());

## Understanding Hyperparameters

### Max depth, min samples split, min samples leaf

In [ ]:
def report_model(model):
    model_pred = model.predict(X_test)
    model_train_pred = model.predict(X_train)
    print('\n')
    print("Test Set")
    print(confusion_matrix(y_test, model_pred))
    print('\n')
    print(classification_report(y_test,model_pred))
    print('\n')
    print("Train Set")
    print(confusion_matrix(y_train, model_train_pred))
    print('\n')
    print(classification_report(y_train,model_train_pred))
    plt.figure(figsize=(12,8),dpi=100)
    plot_tree(model,filled=True, feature_names=X.columns, class_names=df.species.unique());

In [ ]:
pruned_tree = DecisionTreeClassifier(max_depth=2, random_state=101)#max_depph 2 katman ağaç olarak dallanmasını istiyoruz
pruned_tree.fit(X_train,y_train)                                   #none olursa hiç bir hataya yer olmayana kadar dallanmaya devam et
#eğer böylwe yaparsak overfir olur train setine hata yaptırmak istiyoruz bias eklemeliyiz
#kırparak yapıyoruz max depth ile oynayarak bunu yapıyoruz train setinin scorunu 
#kötüleştirip test setini iyileştirmeye çalışıyoruz en fazla kullanılan max_dephtir
#min_sample_split bölünmeye devam etmesi için gereken şart max_depth =None diyoruz 
#min_samples_leaf bir yaprağın yaprak olabilmesi için minimum kaç tane gözlem olması gerektiği
#bu durumlar ilede 

In [ ]:
report_model(pruned_tree,)

### Max Leaf Nodes

In [ ]:
pruned_tree_2 = DecisionTreeClassifier(max_leaf_nodes=5, random_state=101,)#en sonda kalan yaprak sayımı ayarlıyor
pruned_tree_2.fit(X_train,y_train)

In [ ]:
report_model(pruned_tree_2)

## Criterion

In [ ]:
entropy_tree = DecisionTreeClassifier(criterion='entropy', random_state=101,)
entropy_tree.fit(X_train,y_train)

In [ ]:
report_model(entropy_tree)

## Max_features, Splitter

In [ ]:
tree = DecisionTreeClassifier(splitter = "best", max_features=2) # en iyi feature olarak onu seçiyor random için modelin sorduğu
#sorular artar best içinde modeli overfite götüren özellikler arasında 
#max_feature=3 dersek 9 feature içinden  rastgele 3 seçer o featurelar içinden en iyi ayrımı seçip yapar
#max_feature ile hepsini kullanmasını istemiyorum random olarak seçip
tree.fit(X_train,y_train)

In [ ]:
report_model(tree)

## Find Best Parameters

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
model = DecisionTreeClassifier(random_state=101)

In [ ]:
param_grid = {"splitter":["best", "random"],                         #auto feature karekökünü alır
              "max_features":[None, "auto", "log2", 2, 3,4, 5,6, 7], #log2 tüm bunlar overfitten kaçınmak için 
              "max_depth": [None, 2,3, 4, 5],
              "min_samples_leaf": [1, 2, 3, 4, 5, 6,7],
              "min_samples_split": [2, 3, 5, 6, 7,8,9]}

In [ ]:
grid_model = GridSearchCV(estimator=model,
                          param_grid=param_grid,
                          scoring='f1_micro',
                          cv=10,
                          n_jobs = -1)

In [ ]:
grid_model.fit(X_train2, y_train2)

In [ ]:
grid_model.best_params_

In [ ]:
eval_metric(DT2_model, X_train2, y_train2, X_test2, y_test2)

In [ ]:
print("yavuz")